In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
# Load the data
athlete_df = pd.read_csv('../Resources/clean_athletes.csv')
athlete_df = athlete_df.drop('Unnamed: 0', axis=1)
athlete_df.head()

,ID,Name,Sex,Age,Height,Weight,Country_Name,Country_Code,Games,Year,Season,City,Sport,Event,Medal
0,22,Andreea Aanei,F,22.0,170.0,125.0,Romania,ROU,2016 Summer,2016,Summer,Rio de Janeiro,Weightlifting,Weightlifting Women's Super-Heavyweight,NaN
1,51,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Individual All-Around,NaN
2,51,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Floor Exercise,NaN
3,51,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Parallel Bars,NaN
4,51,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Horizontal Bar,NaN


In [3]:
# Fill in rows with missing medals
athlete_df['Medal'] = athlete_df['Medal'].fillna('No_Medals')
athlete_df.head()

,ID,Name,Sex,Age,Height,Weight,Country_Name,Country_Code,Games,Year,Season,City,Sport,Event,Medal
0,22,Andreea Aanei,F,22.0,170.0,125.0,Romania,ROU,2016 Summer,2016,Summer,Rio de Janeiro,Weightlifting,Weightlifting Women's Super-Heavyweight,No_Medals
1,51,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Individual All-Around,No_Medals
2,51,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Floor Exercise,No_Medals
3,51,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Parallel Bars,No_Medals
4,51,Nstor Abad Sanjun,M,23.0,167.0,64.0,Spain,ESP,2016 Summer,2016,Summer,Rio de Janeiro,Gymnastics,Gymnastics Men's Horizontal Bar,No_Medals


In [4]:
# Drop the non-beneficial column 'Name'
athlete_df = athlete_df.drop(['ID', 'Name', 'Country_Name', 'Games', 'Year', 'City', 'Event'],1)
athlete_df.head()

C:\Users\ponen\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,Sex,Age,Height,Weight,Country_Code,Season,Sport,Medal
0,F,22.0,170.0,125.0,ROU,Summer,Weightlifting,No_Medals
1,M,23.0,167.0,64.0,ESP,Summer,Gymnastics,No_Medals
2,M,23.0,167.0,64.0,ESP,Summer,Gymnastics,No_Medals
3,M,23.0,167.0,64.0,ESP,Summer,Gymnastics,No_Medals
4,M,23.0,167.0,64.0,ESP,Summer,Gymnastics,No_Medals


In [5]:
# Encode Medals
medals = {'Gold': 3, 'Silver': 2, 'Bronze': 1, 'No_Medals':0}
athlete_df['Medal'] = athlete_df['Medal'].map(medals)
athlete_df.head()

,Sex,Age,Height,Weight,Country_Code,Season,Sport,Medal
0,F,22.0,170.0,125.0,ROU,Summer,Weightlifting,0
1,M,23.0,167.0,64.0,ESP,Summer,Gymnastics,0
2,M,23.0,167.0,64.0,ESP,Summer,Gymnastics,0
3,M,23.0,167.0,64.0,ESP,Summer,Gymnastics,0
4,M,23.0,167.0,64.0,ESP,Summer,Gymnastics,0


In [6]:
athlete_df.columns

Index(['Sex', 'Age', 'Height', 'Weight', 'Country_Code', 'Season', 'Sport',
       'Medal'],
      dtype='object')

In [7]:
# Sum number of Medals per Country
medal_count_by_country = athlete_df.loc[:, ['Sport', 'Country_Code', 'Medal']]
medal_count_by_country

,Sport,Country_Code,Medal
0,Weightlifting,ROU,0
1,Gymnastics,ESP,0
2,Gymnastics,ESP,0
3,Gymnastics,ESP,0
4,Gymnastics,ESP,0
...,...,...,...
271111,Athletics,GRE,0
271112,Gymnastics,SUI,2
271113,Gymnastics,SUI,2
271114,Gymnastics,SUI,0


In [8]:
# Group the data by Country_Code and Sport and calculate the sum of the medals columns
medals = medal_count_by_country.groupby(['Sport', 'Country_Code', 'Medal']).size().reset_index(name='Count')

medals.head()



,Sport,Country_Code,Medal,Count
0,Aeronautics,SUI,3,1
1,Alpine Skiing,ALB,0,7
2,Alpine Skiing,ALG,0,10
3,Alpine Skiing,AND,0,105
4,Alpine Skiing,ARG,0,262


In [9]:
SUI_test = medals[((medals['Sport'] == 'Curling') & (medals['Country_Code'] == 'SUI'))]
SUI_test.head()

,Sport,Country_Code,Medal,Count
2047,Curling,SUI,0,20
2048,Curling,SUI,1,9
2049,Curling,SUI,2,9
2050,Curling,SUI,3,5


In [11]:
medals

,Sport,Country_Code,Medal,Count
0,Aeronautics,SUI,3,1
1,Alpine Skiing,ALB,0,7
2,Alpine Skiing,ALG,0,10
3,Alpine Skiing,AND,0,105
4,Alpine Skiing,ARG,0,262
...,...,...,...,...
7083,Wrestling,YUG,0,63
7084,Wrestling,YUG,1,6
7085,Wrestling,YUG,2,6
7086,Wrestling,YUG,3,4


In [14]:
# Preprocess the data

In [15]:
merged_df = pd.merge(athlete_df, medals, on =['Sport', 'Country_Code'])
#merged_df = merged_df.drop('Country_Code', axis=1)

merged_df.head()

,Sex,Age,Height,Weight,Country_Code,Season,Sport,Medal_x,Medal_y,Count
0,F,22.0,170.0,125.0,ROU,Summer,Weightlifting,0,0,58
1,F,22.0,170.0,125.0,ROU,Summer,Weightlifting,0,1,4
2,F,22.0,170.0,125.0,ROU,Summer,Weightlifting,0,2,7
3,F,22.0,170.0,125.0,ROU,Summer,Weightlifting,0,3,2
4,M,27.0,168.0,79.0,ROU,Summer,Weightlifting,0,0,58


In [16]:
# Drop the rows with missing values in the 'Height' column
#athlete_df.dropna(subset=['Height'], inplace=True)
#athlete_df.head()

In [17]:
# Drop the rows with missing values in the 'Weight' column
#athlete_df.dropna(subset=['Weight'], inplace=True)
#athlete_df.head()

In [18]:
print("Number of rows:", merged_df.shape[0])

Number of rows: 848765


In [19]:
# Convert the 'Sex' column to binary values
merged_df['Sex'] = merged_df['Sex'].map({'M':1, 'F':0})
merged_df.head()

,Sex,Age,Height,Weight,Country_Code,Season,Sport,Medal_x,Medal_y,Count
0,0,22.0,170.0,125.0,ROU,Summer,Weightlifting,0,0,58
1,0,22.0,170.0,125.0,ROU,Summer,Weightlifting,0,1,4
2,0,22.0,170.0,125.0,ROU,Summer,Weightlifting,0,2,7
3,0,22.0,170.0,125.0,ROU,Summer,Weightlifting,0,3,2
4,1,27.0,168.0,79.0,ROU,Summer,Weightlifting,0,0,58


In [20]:
# Convert the 'Season' column to binary values
merged_df['Season'] = merged_df['Season'].map({'Summer':1, 'Winter':0})
merged_df.head()

,Sex,Age,Height,Weight,Country_Code,Season,Sport,Medal_x,Medal_y,Count
0,0,22.0,170.0,125.0,ROU,1,Weightlifting,0,0,58
1,0,22.0,170.0,125.0,ROU,1,Weightlifting,0,1,4
2,0,22.0,170.0,125.0,ROU,1,Weightlifting,0,2,7
3,0,22.0,170.0,125.0,ROU,1,Weightlifting,0,3,2
4,1,27.0,168.0,79.0,ROU,1,Weightlifting,0,0,58


In [21]:
# Encode the 'Sport' column using one-hot encoding
merged_df = pd.get_dummies(merged_df, columns=['Sport'])
merged_df = merged_df.drop('Country_Code', axis=1)
merged_df.head()

,Sex,Age,Height,Weight,Season,Medal_x,Medal_y,Count,Sport_Aeronautics,Sport_Alpine Skiing,...,Sport_Table Tennis,Sport_Taekwondo,Sport_Tennis,Sport_Trampolining,Sport_Triathlon,Sport_Tug-Of-War,Sport_Volleyball,Sport_Water Polo,Sport_Weightlifting,Sport_Wrestling
0,0,22.0,170.0,125.0,1,0,0,58,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,22.0,170.0,125.0,1,0,1,4,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,22.0,170.0,125.0,1,0,2,7,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,22.0,170.0,125.0,1,0,3,2,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,27.0,168.0,79.0,1,0,0,58,0,0,...,0,0,0,0,0,0,0,0,1,0


In [22]:
# Print the names of the new columns
print(merged_df.columns)

Index(['Sex', 'Age', 'Height', 'Weight', 'Season', 'Medal_x', 'Medal_y',
       'Count', 'Sport_Aeronautics', 'Sport_Alpine Skiing', 'Sport_Alpinism',
       'Sport_Archery', 'Sport_Art Competitions', 'Sport_Athletics',
       'Sport_Badminton', 'Sport_Baseball', 'Sport_Basketball',
       'Sport_Basque Pelota', 'Sport_Beach Volleyball', 'Sport_Biathlon',
       'Sport_Bobsleigh', 'Sport_Boxing', 'Sport_Canoeing', 'Sport_Cricket',
       'Sport_Croquet', 'Sport_Cross Country Skiing', 'Sport_Curling',
       'Sport_Cycling', 'Sport_Diving', 'Sport_Equestrianism', 'Sport_Fencing',
       'Sport_Figure Skating', 'Sport_Football', 'Sport_Freestyle Skiing',
       'Sport_Golf', 'Sport_Gymnastics', 'Sport_Handball', 'Sport_Hockey',
       'Sport_Ice Hockey', 'Sport_Jeu De Paume', 'Sport_Judo',
       'Sport_Lacrosse', 'Sport_Luge', 'Sport_Military Ski Patrol',
       'Sport_Modern Pentathlon', 'Sport_Motorboating',
       'Sport_Nordic Combined', 'Sport_Polo', 'Sport_Racquets',
       'Sport

In [23]:
# Print the index of a new sport column - 'Sport_Swimming' for example
#index = athlete_df.columns.get_loc('Sport_Swimming')
#print(index)

In [24]:
# Remove any remaining rows with missing values
merged_df.dropna(inplace=True)
merged_df.tail()

,Sex,Age,Height,Weight,Season,Medal_x,Medal_y,Count,Sport_Aeronautics,Sport_Alpine Skiing,...,Sport_Table Tennis,Sport_Taekwondo,Sport_Tennis,Sport_Trampolining,Sport_Triathlon,Sport_Tug-Of-War,Sport_Volleyball,Sport_Water Polo,Sport_Weightlifting,Sport_Wrestling
848547,1,25.0,170.0,75.0,1,1,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
848548,1,25.0,170.0,75.0,1,1,3,8,0,0,...,0,0,0,0,0,1,0,0,0,0
848572,1,24.0,185.0,81.0,1,0,0,5,0,0,...,0,0,0,0,0,1,0,0,0,0
848573,1,24.0,185.0,81.0,1,0,2,8,0,0,...,0,0,0,0,0,1,0,0,0,0
848609,1,21.0,170.0,57.0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# Split the data into features and target
X= merged_df.drop('Age', axis=1)
y= merged_df['Age']

In [26]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(merged_df.drop('Age', axis=1), merged_df['Age'], test_size=0.2, random_state=42)

In [27]:
# Define the model
rf = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
# Train the model
rf.fit(X_train, y_train)

In [ ]:
# Make predictions on the test set
y_pred = rf.predict(X_test)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: {:.2f}%'.format(accuracy * 100))

In [ ]:
merged_df.columns

In [ ]:
# Predicting an athelete's potential medal win

In [ ]:
# Define the input data for a new athlete to predict which medal_category, if any, they will win
new_athlete_data = pd.DataFrame({'Sex': ['0'], 'Age': [25], 'Height':[170], 'Weight':[110],'Season':[0],'Gold':[0],
       'No_Medals':[0], 'Silver':[0], 'Bronze_Prob':[0], 'Silver_Prob':[0], 'Gold_Prob':[0],
       'No_Medals_Prob':[0], 'Sport_Aeronautics': [0],
       'Sport_Alpine Skiing': [0], 'Sport_Alpinism': [0], 'Sport_Archery': [0],
       'Sport_Art Competitions': [0], 'Sport_Athletics': [0], 'Sport_Badminton': [0],
       'Sport_Baseball': [0], 'Sport_Basketball': [0], 'Sport_Basque Pelota': [0],
       'Sport_Beach Volleyball': [0], 'Sport_Biathlon': [0], 'Sport_Bobsleigh': [0],
       'Sport_Boxing': [0], 'Sport_Canoeing': [0], 'Sport_Cricket': [0], 'Sport_Croquet': [0],
       'Sport_Cross Country Skiing': [0], 'Sport_Curling': [0], 'Sport_Cycling': [0],
       'Sport_Diving': [0], 'Sport_Equestrianism': [0], 'Sport_Fencing': [0],
       'Sport_Figure Skating': [0], 'Sport_Football': [0], 'Sport_Freestyle Skiing': [0],
       'Sport_Golf': [0], 'Sport_Gymnastics': [0], 'Sport_Handball': [0], 'Sport_Hockey': [0],
       'Sport_Ice Hockey': [0], 'Sport_Jeu De Paume': [0], 'Sport_Judo': [0],
       'Sport_Lacrosse': [0], 'Sport_Luge': [0], 'Sport_Military Ski Patrol': [0],
       'Sport_Modern Pentathlon': [0], 'Sport_Motorboating': [0],
       'Sport_Nordic Combined': [0], 'Sport_Polo': [0], 'Sport_Racquets': [0],
       'Sport_Rhythmic Gymnastics': [0], 'Sport_Roque': [0], 'Sport_Rowing': [0],
       'Sport_Rugby': [0], 'Sport_Rugby Sevens': [0], 'Sport_Sailing': [0], 'Sport_Shooting': [0],
       'Sport_Short Track Speed Skating': [0], 'Sport_Skeleton': [0],
       'Sport_Ski Jumping': [0], 'Sport_Snowboarding': [0], 'Sport_Softball': [0],
       'Sport_Speed Skating': [0], 'Sport_Swimming': [0], 'Sport_Synchronized Swimming': [0],
       'Sport_Table Tennis': [0], 'Sport_Taekwondo': [0], 'Sport_Tennis': [0],
       'Sport_Trampolining': [0], 'Sport_Triathlon': [0], 'Sport_Tug-Of-War': [0],
       'Sport_Volleyball': [0], 'Sport_Water Polo': [0], 'Sport_Weightlifting': [0],
       'Sport_Wrestling': [0]})

In [ ]:
# Use the trained model to predict the medal category for the new athlete
prediction = rf.predict(new_athlete_data)

# Print the predicted medal category
print(f"Medal prediction for new athlete: {prediction}")